In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("/content/A_Z Handwritten Data.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

# Split into data and labels 
alpha_data = dataset.drop('label',axis = 1)
alpha_labels = dataset['label']

In [3]:
# The below step is to make sure both mnist and the alphabet dataset are of the same dimension.
# The standard mnist dataset has 3 dimensions while alphabet has 2. So a separate dataset was used from
# Kaggle where the dataset was split into train and test. They have been combined in the below code.

dataset = pd.read_csv("/content/mnist_train.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

digit_data1 = dataset.drop('label',axis = 1)
digits_labels1 = dataset['label']

dataset = pd.read_csv("/content/mnist_test.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

digit_data2 = dataset.drop('label',axis = 1)
digits_labels2 = dataset['label']

# append both the datasets together
digit_data = digit_data1.append(digit_data2)
digits_labels = digits_labels1.append(digits_labels2)

In [4]:
# Change the labels to avoid overlapping with the digit dataset
alpha_labels += 10

#combine both the datasets into a single dataset
data = np.vstack([alpha_data, digit_data])
labels = np.hstack([alpha_labels, digits_labels])

#expand the dimensions and normalise the data
data = np.expand_dims(data, axis=-1)
data = data / 255.0

# Splitting the dataset below
(x_train, x_test, y_train, y_test) =  train_test_split(data, labels, test_size=0.25, random_state=42)

In [5]:
# Reshape the data into 3 dimensions. keras.backend is used to adjust teh data based on the channel ordering

import tensorflow.keras.backend as K

if K.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, 28,28)
  x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
  input_shape = (1, 28, 28)
else:
  x_train = x_train.reshape(x_train.shape[0],28, 28, 1)
  x_test = x_test.reshape(x_test.shape[0],28, 28, 1)
  input_shape = (28, 28, 1)

In [6]:
# Image augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
) 

In [18]:
# A typical deep learning model with two middle layers.

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(36, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
model.fit(
    x_train,
    y_train,
    epochs=10
)

Epoch 1/10
10370/10370 [==============================] - 24s 2ms/step - loss: 0.5246 - accuracy: 0.8566
Epoch 2/10
10370/10370 [==============================] - 22s 2ms/step - loss: 0.1614 - accuracy: 0.9530
Epoch 3/10
10370/10370 [==============================] - 22s 2ms/step - loss: 0.1275 - accuracy: 0.9623
Epoch 4/10
10370/10370 [==============================] - 22s 2ms/step - loss: 0.1070 - accuracy: 0.9684
Epoch 5/10
10370/10370 [==============================] - 22s 2ms/step - loss: 0.0941 - accuracy: 0.9719
Epoch 6/10
10370/10370 [==============================] - 23s 2ms/step - loss: 0.0873 - accuracy: 0.9737
Epoch 7/10
10370/10370 [==============================] - 23s 2ms/step - loss: 0.0800 - accuracy: 0.9757
Epoch 8/10
10370/10370 [==============================] - 23s 2ms/step - loss: 0.0760 - accuracy: 0.9770
Epoch 9/10
10370/10370 [==============================] - 25s 2ms/step - loss: 0.0710 - accuracy: 0.9785
Epoch 10/10
10370/10370 [==============================

In [21]:
loss, accuracy = model.evaluate(x_test, y_test)
print(accuracy*100)
print(loss)

3457/3457 [==============================] - 4s 1ms/step - loss: 0.1303 - accuracy: 0.9702
97.01752662658691
0.13025005161762238


In [23]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('7.png')
	# predict the class
	digit = model.predict_classes(img)
	print(digit[0])
 
# entry point, run the example
run_example()

7
